In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import string
from nltk.stem import LancasterStemmer

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dazad\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dazad\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dazad\AppData\Roaming\nltk_data...


True

In [7]:
df = pd.read_csv("tweets.csv", delimiter=',' ,header = None ,engine='python')

In [8]:
df

,0,1,2
0,id,author,status
1,1,Donald J. Trump,I will be making a major statement from the @W...
2,2,Donald J. Trump,Just arrived at #ASEAN50 in the Philippines fo...
3,3,Donald J. Trump,"After my tour of Asia, all Countries dealing w..."
4,4,Donald J. Trump,Great to see @RandPaul looking well and back o...
...,...,...,...
396,396,Justin Trudeau,RT @googlecanada: Watch tmw: @JustinTrudeau di...
397,397,Justin Trudeau,"Today in Ottawa, I met with the Modern Treaty ..."
398,398,Justin Trudeau,Voici le sommaire de ma rencontre avec les Pre...
399,399,Justin Trudeau,RT @VeteransENG_CA: #Day1 of 11 Days of Rememb...


In [9]:
df.columns = ["id" ,"Autor","Estado"]
df = df.drop(0)
df.head()

,id,Autor,Estado
1,1,Donald J. Trump,I will be making a major statement from the @W...
2,2,Donald J. Trump,Just arrived at #ASEAN50 in the Philippines fo...
3,3,Donald J. Trump,"After my tour of Asia, all Countries dealing w..."
4,4,Donald J. Trump,Great to see @RandPaul looking well and back o...
5,5,Donald J. Trump,Excited to be heading home to see the House pa...


In [10]:
df.duplicated().sum()

0

In [11]:
df.drop_duplicates(inplace = True)

In [13]:
df.isna().sum()

id        0
Autor     0
Estado    0
dtype: int64

In [14]:
## Función para el procesamiento de datos
stopword = set(stopwords.words('english'))
stemmer = LancasterStemmer()
def preprocessing(text):
    # print(text)
    try:
      # Convert text to lowercase
      text = text.lower()
      text = re.sub('-',' ',text)   # replace `word-word` as `word word`
      text = re.sub(f'[{string.digits}]',' ',text)  # remove digits
      text = ' '.join([stemmer.stem(word) for word in text.split() if word not in stopword])  # remove stopwords and stem other words
      text =  re.sub(r'@\S+', '',text)                     # remove twitter handles
      text =  re.sub(r'http\S+', '',text)                  # remove urls
      text =  re.sub(r'pic.\S+', '',text)
      text =  re.sub(r"[^a-zA-Z+']", ' ',text)             # only keeps characters
      text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text+' ')      # keep words with length>1 only
      text = ''.join([i for i in text if i not in string.punctuation])
      words = nltk.tokenize.word_tokenize(text,language="english", preserve_line=True)
      text = " ".join([i for i in words if i not in stopword and len(i)>2])
      text= re.sub("\s[\s]+", " ",text).strip()            # remove repeated/leading/trailing spaces
    except:
      print('Error:')
      print(text)

    return re.sub(f'[{re.escape(string.punctuation)}]','',text) # remove punctuations

In [15]:
df["Estado_clean"] =  df["Estado"].apply(preprocessing)

In [16]:
df.head()

,id,Autor,Estado,Estado_clean
1,1,Donald J. Trump,I will be making a major statement from the @W...,mak maj stat upon return tim dat set
2,2,Donald J. Trump,Just arrived at #ASEAN50 in the Philippines fo...,asean philippin fin stop world leaders lead fa...
3,3,Donald J. Trump,"After my tour of Asia, all Countries dealing w...",tour asia country deal trad know rul changed u...
4,4,Donald J. Trump,Great to see @RandPaul looking well and back o...,gre see look wel back sen floor help tax cut r...
5,5,Donald J. Trump,Excited to be heading home to see the House pa...,excit head hom see hous pass gre tax bil middl...


In [17]:
df["Autor"].value_counts()

Autor
Donald J. Trump    200
Justin Trudeau     200
Name: count, dtype: int64

In [18]:
data = df.drop(["id","Estado"] , axis = 1) # will drop column
data.head()

,Autor,Estado_clean
1,Donald J. Trump,mak maj stat upon return tim dat set
2,Donald J. Trump,asean philippin fin stop world leaders lead fa...
3,Donald J. Trump,tour asia country deal trad know rul changed u...
4,Donald J. Trump,gre see look wel back sen floor help tax cut r...
5,Donald J. Trump,excit head hom see hous pass gre tax bil middl...


In [19]:
x = df["Estado_clean"]
y = df["Autor"]

In [20]:
x_l = list(x)
y_l = list(y)

In [21]:
x_train ,x_test ,y_train ,y_test = train_test_split(x_l ,y_l ,test_size = 0.2, random_state=50)

In [22]:
print(len(x_l))
print(len(x_train))

400
320


In [23]:
vectorize = CountVectorizer()
x_train1 = vectorize.fit_transform(x_train)
x_test1 =  vectorize.transform(x_test)

In [24]:
mnb = MultinomialNB()
mnb.fit(x_train1 ,y_train)
y_pred = mnb.predict(x_test1)

In [25]:
metrics.accuracy_score(y_pred,y_test)

0.9

In [26]:
random_forest = RandomForestClassifier()

# Fit the model with GridSearchCV
random_forest.fit(x_train1, y_train)
y_pred = random_forest.predict(x_test1)

In [27]:
accuracy_score(y_test, y_pred)

0.9375

In [28]:
#hello
nuevos_datos = ["John meets with The Director, the head of the Ruska Roma crime syndicate, to whom he presents a rosary and demands safe passage to Casablanca. As John was once a member, the Director reluctantly agrees. It's also revealed that John Wick's birth name was Jardani Jovonovich. Meanwhile, a High Table Adjudicator meets with New York Continental manager Winston and the Bowery King, notifying them that they both have seven days to resign from their positions as a consequence of helping John.[a] The Adjudicator enlists Zero, a Japanese assassin, and orders him to stab the Director through both of her hands as penance for aiding John. In Casablanca, John meets Sofia, his former friend and manager of the Moroccan Continental."]
nuevos_datos[0] = preprocessing(nuevos_datos[0])
nuevos_datos[0]

'john meet director head rusk rom crim syndicate pres ros demand saf pass casablanca john member direct reluct agrees also rev john wicks bir nam jardan jovonovich meanwhile high tabl adjud meet new york contin man winston bowery king sev day resign posit consequ help john adjud enl zero japanes assassin ord stab direct hand pen aid john casablanca john meet sofia form friend man morocc continental'

In [29]:
X_nuevos = vectorize.transform(nuevos_datos)

In [30]:
# Para el modelo Multinomial Naive Bayes
predicciones_nb_nuevos = mnb.predict(X_nuevos)

# Para el modelo Random Forest
predicciones_rf_nuevos = random_forest.predict(X_nuevos)

In [31]:
# Para el modelo Multinomial Naive Bayes
for i, prediccion in enumerate(predicciones_nb_nuevos):
    print(f"Nueva entrada {i + 1}: {prediccion}")

# Para el modelo Random Forest
for i, prediccion in enumerate(predicciones_rf_nuevos):
    print(f"Nueva entrada {i + 1}: {prediccion}")

Nueva entrada 1: Donald J. Trump
Nueva entrada 1: Donald J. Trump


In [32]:
import joblib
joblib.dump(random_forest, 'random_forest.pkl')
joblib.dump(vectorize, 'preprocesamiento.pkl')


['preprocesamiento.pkl']

In [33]:
modelo_cargado = joblib.load('random_forest.pkl')
vectorizer_cargado = joblib.load('preprocesamiento.pkl')

modelo_cargado.predict(X_nuevos)

array(['Donald J. Trump'], dtype='<U15')

In [67]:
## Función para el procesamiento de datos
stopword = set(stopwords.words('english'))
stemmer = LancasterStemmer()
def preprocessing(text):
    # print(text)
    try:
      # Convert text to lowercase
      text = text.lower()
      text = re.sub('-',' ',text)   # replace `word-word` as `word word`
      text = re.sub(f'[{string.digits}]',' ',text)  # remove digits
      text = ' '.join([stemmer.stem(word) for word in text.split() if word not in stopword])  # remove stopwords and stem other words
      text =  re.sub(r'@\S+', '',text)                     # remove twitter handles
      text =  re.sub(r'http\S+', '',text)                  # remove urls
      text =  re.sub(r'pic.\S+', '',text)
      text =  re.sub(r"[^a-zA-Z+']", ' ',text)             # only keeps characters
      text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text+' ')      # keep words with length>1 only
      text = ''.join([i for i in text if i not in string.punctuation])
      words = nltk.tokenize.word_tokenize(text,language="english", preserve_line=True)
      text = " ".join([i for i in words if i not in stopword and len(i)>2])
      text= re.sub("\s[\s]+", " ",text).strip()            # remove repeated/leading/trailing spaces
    except:
      print('Error:')
      print(text)

    return re.sub(f'[{re.escape(string.punctuation)}]','',text) # remove punctuations
nuevos_datos = ["John meets with The Director, the head of the Ruska Roma crime syndicate, to whom he presents a rosary and demands safe passage to Casablanca. As John was once a member, the Director reluctantly agrees. It's also revealed that John Wick's birth name was Jardani Jovonovich. Meanwhile, a High Table Adjudicator meets with New York Continental manager Winston and the Bowery King, notifying them that they both have seven days to resign from their positions as a consequence of helping John.[a] The Adjudicator enlists Zero, a Japanese assassin, and orders him to stab the Director through both of her hands as penance for aiding John. In Casablanca, John meets Sofia, his former friend and manager of the Moroccan Continental."]
nuevos_datos[0] = preprocessing(nuevos_datos[0])
nuevos_datos[0]
X_nuevos = vectorizer_cargado.transform(nuevos_datos)
modelo_cargado.predict(X_nuevos)

array(['Donald J. Trump'], dtype='<U15')

In [69]:
!pip show joblib

Name: joblib
Version: 1.3.2
Summary: Lightweight pipelining with Python functions
Home-page: 
Author: 
Author-email: Gael Varoquaux <gael.varoquaux@normalesup.org>
License: BSD 3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: imbalanced-learn, librosa, mlxtend, music21, nltk, scikit-learn
